# Binary Sentiment Classification of ratebeer Reviews

In [1]:
import numpy as np
import pandas as pd

Paths to serialization files

In [2]:
# Matt's paths
# pathToBinClassDir = '/home/matt/w266_saved/binaryclassifier'
# pathToWordId      = '/home/matt/w266_saved/binaryclassifier/wordId.npy'
# pathToCheckpoint  = '/home/matt/w266_saved/binaryclassifier/model.ckpt'

# Dave's paths
pathToBinClassDir = '/home/dal7p/project_model/'
pathToWordId      = '/home/dal7p/project_model/ratebeer/wordId.npy'
pathToCheckpoint  = '/home/dal7p/project_model/model.ckpt'

RNN Model Parameters

In [3]:
batchSize = 24
numClasses = 2  # Binary classification
hiddenSize = 50

assert(batchSize % numClasses == 0)

## Word Vectors

In [4]:
from pathlib import Path
import os.path
wordsList = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordsList.npy'))
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load(os.path.join(str(Path.home()), '.kaggle/wordvectors/pretrained_glove/wordVectors.npy'))

Word Vectors have dimension 50

In [5]:
embedding_dimension = 50

## Reviews

Load the Reviews

In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/dal7p/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
# Read the reviews in
review_df = pd.read_csv('./data_prep/rate_beer_binary_medium_v2.csv', encoding='utf8')


Define the size of the train, dev, and test datasets

In [8]:
import math

# Get indicies of the rows in the dataframe for testing
test_lower_index = 0
test_size = review_df.shape[0]

Remove punctuation, lowercase, and then tokenize the reviews.  The tokens need to be lowercase for the embedding lookup.

In [9]:
import re

strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    return word_tokenize(re.sub(strip_special_chars, " ", string.lower()))

review_df['Tokens'] = review_df['Text'].apply(lambda text: cleanSentences(text))

Create the array of input sentences converted to word IDs. 
One extra integer to store the review ID

In [10]:
maxSeqLength = 267 # From EDA

In [11]:
word_ids = np.zeros((test_size, maxSeqLength+2), dtype='int32')

Convert words to word IDs and store in word_ids

In [12]:
from pathlib import Path
word_id_file = Path(pathToWordId)

if not word_id_file.exists():

    sentence_index = 0

    for _, row in review_df.iterrows():

        # Store the review Id for identifying misclassified reviews in testing
        word_index = 0
        word_ids[sentence_index][word_index] = row['Id']   # row['id']          #TODO: FIX LATER -- Fixed?
        word_index = word_index + 1
        word_ids[sentence_index][word_index] = row['Score']
        word_index = word_index + 1

        for word in row['Tokens']:

            try:
                word_ids[sentence_index][word_index] = wordsList.index(word)
            except ValueError:
                word_ids[sentence_index][word_index] = 399999 #Vector for unkown words

            word_index = word_index + 1

            if word_index == maxSeqLength:
                break

        sentence_index = sentence_index + 1

    # Shuffle the word_ids matrix
    np.random.shuffle(word_ids)
    
    # Save the word_ids matrix
    binClassDir = Path(pathToBinClassDir)
    
    if not binClassDir.exists():
        os.mkdir(pathToBinClassDir)
    
    np.save(pathToWordId, word_ids)
else:
    word_ids = np.load(pathToWordId)

Create functions to get the train and test batches

In [13]:
reviewsTestIndex = 0

def getOrderedTestReviews(sectionOffset, sectionSize):
    global reviewsTestIndex
    
    labels = []
    arr = np.zeros([batchSize, maxSeqLength])
    ids = np.zeros(batchSize)
    finished = False
    
    for i in range(batchSize):
        
        if reviewsTestIndex >= sectionSize:
            finished = True
            break;

        num = reviewsTestIndex + sectionOffset
        
        if (word_ids[num, 1] == 1) or (word_ids[num, 1] == 2):
            labels.append([1, 0])
        else: 
            labels.append([0, 1])
        
        arr[i] = word_ids[num, 2:]
        ids[i] = word_ids[num, 0]
        reviewsTestIndex += 1
        
    if reviewsTestIndex == sectionSize:
        finished = True
        
    return arr, labels, ids, finished

def resetDevTestIndicies():
    global reviewsTestIndex

    reviewsTestIndex = 0

def getTestBatch():
    return getOrderedTestReviews(test_lower_index, test_size)

## Tensorflow Model

Hyper Parameters

In [14]:
epochs = 100
learning_rate =  0.001
dropout_keep_prob = 0.75
num_layers = 2

Model

In [15]:
import tensorflow as tf

class SentimentGraph:
    
    def __init__(self):
        self.labels = tf.placeholder(tf.float32, [batchSize, numClasses])
        self.input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])
        self.prediction = None
        self.accuracy = None
        self.loss = None
        self.optimizer = None

    def MakeFancyRNNCell(self, H, keep_prob, num_layers=1):
        cells = []
        for _ in range(num_layers):
          cell = tf.nn.rnn_cell.BasicLSTMCell(H, forget_bias=0.0)
          cell = tf.nn.rnn_cell.DropoutWrapper(
              cell, input_keep_prob=keep_prob, output_keep_prob=keep_prob)
          cells.append(cell)
        return tf.nn.rnn_cell.MultiRNNCell(cells)

    def CreateGraph(self):
        data = tf.Variable(tf.zeros([batchSize, maxSeqLength, embedding_dimension]), dtype=tf.float32)
        data = tf.nn.embedding_lookup(wordVectors, self.input_data)

        lstmCell = self.MakeFancyRNNCell(hiddenSize, dropout_keep_prob, num_layers)
        initial_h_ = lstmCell.zero_state(batchSize, dtype=tf.float32)
        
        rnn_out, _ = tf.nn.dynamic_rnn(lstmCell,
                                       data,
                                       initial_state=initial_h_,
                                       dtype=tf.float32)

        W_out = tf.Variable(tf.truncated_normal([hiddenSize, numClasses]), dtype=tf.float32)
        b_out = tf.Variable(tf.constant(0.1, shape=[numClasses]), dtype=tf.float32)

        # Get the output of the last RNN cell
        rnn_out = tf.transpose(rnn_out, [1, 0, 2])
        last_cell_out = tf.gather(rnn_out, int(rnn_out.get_shape()[0]) - 1)

        # Calculate logits
        logits = (tf.matmul(last_cell_out, W_out) + b_out)

        # Calculate prediction and accuracy
        self.prediction = tf.argmax(logits,1)
        correctPred = tf.equal(self.prediction, tf.argmax(self.labels,1))
        self.accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

        # Loss function and optimizer
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=self.labels))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)

/home/dal7p/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Examine the Mispredicted_IMDB_Classification.csv file for error analysis.

In [16]:
def TestModelTest(session, graph):

    # Support for saving mispredicted reviews
    csv = open('Mispredicted_ratebeer_Classification.csv', 'w')
    csv.write("Id\n")
    
    i = 0  # Must stay outside the loops
    
    accuracy_measurements = []
    loss_measurements = []
    finished = False
    
    while not finished:
    
        nextBatch, nextBatchLabels, reviewIds, finished = getTestBatch()

        # For ease of implementation, just skip partially filled batches
        if not finished:
            feed_dict = {
                graph.input_data: nextBatch,
                graph.labels: nextBatchLabels
            }

            accuracy_, loss_ = sess.run([graph.accuracy, graph.loss], feed_dict)
            
            i += 1

            accuracy_measurements.append(accuracy_)
            loss_measurements.append(loss_)
            
            # Write out mispredictions (review IDs) to a .csv file
            if accuracy_ < 1.0:
                
                predictions_ = sess.run(graph.prediction, feed_dict)

                for index in range(len(predictions_)):

                    if predictions_[index] != np.argmax(nextBatchLabels[index]):
                        csv.write(str(int(reviewIds[index])) + "\n")

    print('Testing (Test) Results:')
    print('The average accuracy is: ', np.mean(accuracy_measurements))
    print('The average loss is: ', np.mean(loss_measurements))

### Run prior to testing

In [17]:
import datetime

logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"

Use the following:
tensorboard --logdir=tensorboard

### Run prior to testing

In [18]:
resetDevTestIndicies()

## Test

In [19]:
tf.reset_default_graph()
    
with tf.Session() as sess:

    # Create the graph
    sentimentGraph = SentimentGraph()
    sentimentGraph.CreateGraph()
    
    sess.run(tf.global_variables_initializer())
    
    saver = tf.train.Saver()
    saver.restore(sess, pathToCheckpoint)

    # Test model
    TestModelTest(sess, sentimentGraph)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

INFO:tensorflow:Restoring parameters from /home/dal7p/project_model/model.ckpt
Testing (Test) Results:
The average accuracy is:  0.66222835
The average loss is:  0.7247493
